In [1]:
'''
1. Data generator
    a. Loads vocab
    b. Loads image features
    c. provide data for training.
2. Builds image caption model.
3. Trains the model.
'''

'\n1. Data generator\n    a. Loads vocab\n    b. Loads image features\n    c. provide data for training.\n2. Builds image caption model.\n3. Trains the model.\n'

In [2]:
import os
import sys
import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging
import pprint
import pickle
import numpy as np
import math
import random

In [3]:
input_description_file = './dataset/flickr30k/results_20130124.token'
input_img_feature_dir = './dataset/inception_v3_features_bakup/'
input_vocab_file = './dataset/flickr30k/vocab.txt'
output_dir = './local_run/'

if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)

In [9]:
def get_default_params():
    return tf.contrib.training.HParams(
        num_vocab_word_threshold = 3,
        num_embedding_size = 32,
        num_timesteps = 10,
        num_lstm_nodes = [64, 64],
        num_lstm_layers = 2,
        num_fc_nodes = 32,
        batch_size = 80,
        cell_type = 'lstm',
        clip_lstm_grads = 1.0,
        learning_rate = 0.001,
        keep_prob = 0.8,
        log_frequent = 10,
        save_frequent = 100
    )

hps = get_default_params()

In [5]:
class Vocab(object):
    def __init__(self, filename, num_vocab_word_threshold):
        self._id_to_word = {}
        self._word_to_id = {}
        self._unk = -1
        self._eos = -1
        self._num_vocab_word_threshold = num_vocab_word_threshold
        self._read_dict(filename)
        
    def _read_dict(self, filename):
        with gfile.GFile(filename, 'r') as f:
            lines = f.readlines()
            
        for line in lines:
            word, frequency = line.strip('\n\r').split('\t')
            frequency = int(frequency)
            if frequency < self._num_vocab_word_threshold:
                continue
            idx = len(self._id_to_word)
            if word == '<UNK>':
                self._unk = idx
            elif word == '.':
                self._eos = idx
#             pprint.pprint(self._word_to_id.items())
#             if word in self._id_to_word or self._word_to_id:
#                 raise Exception("duplicate words in vocab. %s" % word)
            self._word_to_id[word] = idx
            self._id_to_word[idx] = word
            
    @property
    def unk(self):
        return self._unk
    
    @property
    def eos(self):
        return self._eos
    
    def word_to_id(self, word):
        return self._word_to_id.get(word, self.unk)
    
    def id_to_word(self, idx):
        return self._id_to_word.get(idx, '<UNK>')
    
    def size(self):
        return len(self._word_to_id)
    
    def encode(self, sentence):
        return [self.word_to_id(word) for word in sentence.strip('\r\t').split(' ')]
    
    def decode(self, sentence_id):
        words = [self.id_to_word(idx) for idx in sentence_id]
        return ' '.join(words)
    
vocab = Vocab(input_vocab_file, hps.num_vocab_word_threshold)
vocab_size = vocab.size()

logging.info("vocab_size: %d", vocab_size)
pprint.pprint(vocab.encode("I am a dreamer . "))
pprint.pprint(vocab.decode([4, 5, 100, 2]))

INFO:tensorflow:vocab_size: 10875
[1494, 4709, 1, 0, 2, 0]
'in the snow .'


In [6]:
def parse_token_file(token_file):
    '''Parses input_description_file to a dict'''
    img_name_to_token = {}
    with gfile.GFile(token_file, 'r') as f:
        lines = f.readlines()
        
    for line in lines:
        img_id, description = line.strip('\n\r').split('\t')
        img_name = img_id.split('#')[0]
        img_name_to_token.setdefault(img_name, [])
        img_name_to_token[img_name].append(description)
        
    return img_name_to_token

def convert_token_to_id(img_name_to_token, vocab):
    '''Converts tokens of each description of img to id'''
    img_name_to_token_id = {}
    for img_name in img_name_to_token:
        img_name_to_token_id.setdefault(img_name, [])
        for description in img_name_to_token[img_name]:
            tokens_ids = vocab.encode(description)
            img_name_to_token_id[img_name].append(tokens_ids)
            
    return img_name_to_token_id

img_name_to_token = parse_token_file(input_description_file)
img_name_to_token_id = convert_token_to_id(img_name_to_token, vocab)

logging.info("num of all images: %d" % len(img_name_to_token))
pprint.pprint(img_name_to_token['2778832101.jpg'])
logging.info("num of all images: %d" % len(img_name_to_token_id))
pprint.pprint(img_name_to_token_id['2778832101.jpg'])

INFO:tensorflow:num of all images: 31783
['A man in jeans is reclining on a green metal bench along a busy sidewalk and '
 'crowded street .',
 'A white male with a blue sweater and gray pants laying on a sidewalk bench .',
 'A man in a blue shirt and gray pants is sleeping on a sidewalk bench .',
 'A person is sleeping on a bench , next to cars .',
 'A man sleeping on a bench in a city area .']
INFO:tensorflow:num of all images: 31783
[[3, 9, 4, 132, 8, 3532, 6, 1, 48, 337, 146, 139, 1, 244, 93, 7, 380, 36, 2],
 [3, 20, 179, 11, 1, 26, 284, 7, 120, 128, 297, 6, 1, 93, 146, 2],
 [3, 9, 4, 1, 26, 21, 7, 120, 128, 8, 340, 6, 1, 93, 146, 2],
 [3, 63, 8, 340, 6, 1, 146, 12, 70, 15, 518, 2],
 [3, 9, 340, 6, 1, 146, 4, 1, 112, 171, 2]]


In [7]:
class ImageCaptionData(object):
    '''Provides data for image caption model.'''
    def __init__(self, 
                 img_name_to_token_id, 
                 img_feature_dir,
                 num_timesteps,
                 vocab,
                 deterministic = False):
        self._vocab = vocab
        self._img_name_to_token_id = img_name_to_token_id
        self._num_timesteps = num_timesteps
        self._deterministic = deterministic
        self._indicator = 0
        
        self._img_feature_filenames = []
        self._img_feature_data = []
        
        self._all_img_feature_filepaths = []
        for filename in gfile.ListDirectory(img_feature_dir):
            self._all_img_feature_filepaths.append(
                os.path.join(img_feature_dir, filename))
        pprint.pprint(self._all_img_feature_filepaths)
        self._load_img_feature_pickle()
        
        if not self._deterministic:
            self._random_shuffle()
            
    def _load_img_feature_pickle(self):
        '''Load img feature data from pickle.'''
        for filepath in self._all_img_feature_filepaths:
            logging.info("loading %s" % filepath)
            with gfile.GFile(filepath, 'rb') as f:
                filenames, features = pickle.load(f)
                self._img_feature_filenames += filenames
                self._img_feature_data.append(features)
        # [#(1000, 1, 1, 2048), #(1000, 1, 1, 2048)] -> #(2000, 1, 1, 2048)
        self._img_feature_data = np.vstack(self._img_feature_data)
        origin_shape = self._img_feature_data.shape
        self._img_feature_data = np.reshape(
            self._img_feature_data,
            (origin_shape[0], origin_shape[-1]))
        self._img_feature_filenames = np.asarray(self._img_feature_filenames)
        print(self._img_feature_data.shape)
        print(self._img_feature_filenames.shape)
        
    def size(self):
        return len(self._img_feature_filenames)
    
    def img_feature_size(self):
        return self._img_feature_data.shape[1]
        
    def _random_shuffle(self):
        '''Shuffle data randomly.'''
        p = np.random.permutation(self.size())
        self._img_feature_filenames = self._img_feature_filenames[p]
        self._img_feature_data = self._img_feature_data[p]
        
    def _img_desc(self, batch_filenames):
        '''Gets description for filenames in batch.'''
        batch_sentence_ids = []
        batch_weights = []
        for filename in batch_filenames:
            token_ids_set = self._img_name_to_token_id[filename]
            chosen_token_ids = random.choice(token_ids_set)
            chosen_token_ids_length = len(chosen_token_ids)
            
            weight = [1 for i in range(chosen_token_ids_length)]
            if chosen_token_ids_length >= self._num_timesteps:
                chosen_token_ids = chosen_token_ids[0: self._num_timesteps]
                weight = weight[0: self._num_timesteps]
            else:
                remaining_length = self._num_timesteps - chosen_token_ids_length
                chosen_token_ids += [self._vocab.eos for i in range(remaining_length)]
                weight += [0 for i in range(remaining_length)]
            batch_sentence_ids.append(chosen_token_ids)
            batch_weights.append(weight)
        batch_sentence_ids = np.asarray(batch_sentence_ids)
        batch_weights = np.asarray(batch_weights)
        return batch_sentence_ids, batch_weights
        
    def next_batch(self, batch_size):
        '''Returns next batch data.'''
        end_indicator = self._indicator + batch_size
        if end_indicator > self.size():
            if not self._deterministic:
                self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self.size()
        
        batch_filenames = self._img_feature_filenames[self._indicator: end_indicator]
        batch_img_features = self._img_feature_data[self._indicator: end_indicator]
        # sentence_ids: [100, 101, 102, 10, 0, 3, 2, 0, 0] -> [1, 1, 1, 1, 0, 1, 1, 0, 0]
        # id为0的weight为0，其他为1
        batch_sentence_ids, batch_weights = self._img_desc(batch_filenames)
        self._indicator = end_indicator
        return batch_img_features, batch_sentence_ids, batch_weights, batch_filenames
    
caption_data = ImageCaptionData(img_name_to_token_id, 
                                input_img_feature_dir,
                                hps.num_timesteps,
                                vocab)
img_feature_dim = caption_data.img_feature_size()
caption_data_size = caption_data.size()
logging.info("img_feature_dim: %d" % img_feature_dim)
logging.info("caption_data_size: %d" % caption_data_size)

batch_img_features, batch_sentence_ids, batch_weights, batch_img_names = caption_data.next_batch(5)
pprint.pprint(batch_img_features)
pprint.pprint(batch_sentence_ids)
pprint.pprint(batch_weights)
pprint.pprint(batch_img_names)

['./dataset/inception_v3_features_bakup/image_feature-0.pickle',
 './dataset/inception_v3_features_bakup/image_feature-1.pickle',
 './dataset/inception_v3_features_bakup/image_feature-10.pickle',
 './dataset/inception_v3_features_bakup/image_feature-11.pickle',
 './dataset/inception_v3_features_bakup/image_feature-12.pickle',
 './dataset/inception_v3_features_bakup/image_feature-13.pickle',
 './dataset/inception_v3_features_bakup/image_feature-14.pickle',
 './dataset/inception_v3_features_bakup/image_feature-15.pickle',
 './dataset/inception_v3_features_bakup/image_feature-16.pickle',
 './dataset/inception_v3_features_bakup/image_feature-17.pickle',
 './dataset/inception_v3_features_bakup/image_feature-18.pickle',
 './dataset/inception_v3_features_bakup/image_feature-19.pickle',
 './dataset/inception_v3_features_bakup/image_feature-2.pickle',
 './dataset/inception_v3_features_bakup/image_feature-20.pickle',
 './dataset/inception_v3_features_bakup/image_feature-21.pickle',
 './dataset/i

In [8]:
def create_rnn_cell(hidden_dim, cell_type):
    '''Returns specific cell according to cell_type.'''
    if cell_type == 'lstm':
        return tf.contrib.rnn.BasicLSTMCell(hidden_dim, 
                                            state_is_tuple = True)
    elif cell_type == 'gru':
        return tf.contrib.rnn.GRUCell(hidden_dim)
    else:
        raise Exception("%s type has not been supported." % cell_type)

def dropout(cell, keep_prob):
    '''Wrap cell with dropout.'''
    return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)

def get_train_model(hps, vocab_size, img_feature_dim):
    num_timesteps = hps.num_timesteps
    batch_size = hps.batch_size
    
    img_feature = tf.placeholder(tf.float32, (batch_size, img_feature_dim))
    sentence = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    mask = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    global_step = tf.Variable(tf.zeros([], tf.int32),
                              name = "global_step",
                              trainable = False)
    
    # prediction process:
    # sentence: [a, b, c, d, e]
    # input: [img, a, b, c, d]
    # img_feature: [0.4, 0.3, 10, 2]
    # prediction1: img_feature -> embedding_img -> lstm -> (a)
    # prediction2: a -> embedding_word -> lstm -> (b)
    # prediction3: b -> embedding_word -> lstm -> (c)
    # ...
    
    # Sets up embedding layer.
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    with tf.variable_scope('embedding', initializer = embedding_initializer):
        embeddings = tf.get_variable('embeddings', [vocab_size, hps.num_embedding_size], tf.float32)
        # embed_token_ids: [batch_size, num_timesteps - 1, num_embedding_size]
        embed_token_ids = tf.nn.embedding_lookup(embeddings, sentence[:, 0:num_timesteps - 1])
    
    img_feature_embed_init = tf.uniform_unit_scaling_initializer(factor = 1.0)
    with tf.variable_scope('img_feature_embed', initializer = img_feature_embed_init):
        # img_feature: [batch_size, img_feature_dim]
        # embed_img: [batch_size, num_embedding_size]
        embed_img = tf.layers.dense(img_feature, hps.num_embedding_size)
        # embed_img: [batch_size, 1, num_embedding_size]
        embed_img = tf.expand_dims(embed_img, 1)
        # embed_inputs: [batch_size, num_timesteps, num_embedding_size]
        embed_inputs = tf.concat([embed_img, embed_token_ids], axis = 1)
    
    # Sets up rnn network
    scale = 1.0 / math.sqrt(hps.num_embedding_size + hps.num_lstm_nodes[-1]) / 3.0
    rnn_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('lstm_rnn', initializer = rnn_init):
        cells = []
        for i in range(hps.num_lstm_layers):
            cell = create_rnn_cell(hps.num_lstm_nodes[i], hps.cell_type)
            cell = dropout(cell, keep_prob)
            cells.append(cell)
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        
        init_state = cell.zero_state(hps.batch_size, tf.float32)
        # run_outputs: [batch_size, num_timesteps, hps.num_lstm_nodes[-1]
        rnn_outputs, _ = tf.nn.dynamic_rnn(cell, 
                                           embed_inputs,
                                           initial_state = init_state)
        
    # Sets up fully-connected layer.
    fc_init = tf.uniform_unit_scaling_initializer(factor = 1.0)
    with tf.variable_scope('fc', initializer = fc_init):
        rnn_outputs_2d = tf.reshape(rnn_outputs, 
                                   [-1, hps.num_lstm_nodes[-1]])
        fc1 = tf.layers.dense(rnn_outputs_2d,
                              hps.num_fc_nodes, 
                              name = 'fc1')
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        fc1_relu = tf.nn.relu(fc1_dropout)
        
        logits = tf.layers.dense(fc1_relu, 
                                 vocab_size, 
                                 name = 'logits')
        
    # Calculates loss
    with tf.variable_scope('loss'):
        sentence_flatten = tf.reshape(sentence, [-1])
        mask_flatten = tf.reshape(mask, [-1])
        mask_flatten_float = tf.cast(mask_flatten, tf.float32)
        mask_sum = tf.reduce_sum(mask_flatten)
        mask_sum_float = tf.cast(mask_sum, tf.float32)
        
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = logits,
            labels = sentence_flatten)
        
        weighted_softmax_loss = tf.multiply(
            softmax_loss, mask_flatten_float)
        loss = tf.reduce_sum(weighted_softmax_loss) / mask_sum_float
        
        prediction = tf.argmax(logits, 1, output_type = tf.int32)
        correct_prediction = tf.equal(prediction, sentence_flatten)
        weighted_correct_prediction = tf.multiply(
            tf.cast(correct_prediction, tf.float32), mask_flatten_float)
        accuracy = tf.reduce_sum(weighted_correct_prediction) / mask_sum_float
        tf.summary.scalar('loss', loss)
    
    # Defines train_op.
    with tf.variable_scope('train_op'):
        tvars = tf.trainable_variables()
        for var in tvars:
            logging.info("variable name: %s", var.name)
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(loss, tvars), hps.clip_lstm_grads)
        optimizer = tf.train.AdamOptimizer(hps.learning_rate)
        train_op = optimizer.apply_gradients(
            zip(grads, tvars), global_step = global_step)
        
    return ((img_feature, sentence, mask, keep_prob),
            (loss, accuracy, train_op),
            global_step)

placeholders, metrics, global_step = get_train_model(hps, vocab_size, img_feature_dim)
img_feature, sentence, mask, keep_prob = placeholders
loss, accuracy, train_op = metrics

summary_op = tf.summary.merge_all()
init_op = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep = 10)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
INFO:tensorflow:variable name: <tf.Variable 'embedding/embeddings:0' shape=(10875, 32) dtype=float32_ref>
INFO:tensorflow:variable name: <tf.Variable 'img_feature_embed/dense/kernel:0' shape=(2048, 32) dtype=float32_ref>
INFO:tensorflow:variable name: <tf.Variable 'img_feature_embed/dense/bias:0' shape=(32,) dtype=float32_ref>
INFO:tensorflow:variable name: <tf.Variable 'lstm_rnn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(96, 256) dtype=float32_ref>
INFO:tensorflow:variable name: <tf.Variable 'lstm_rnn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>
INFO:tensorflow:variable name: <tf.V

In [10]:
training_steps = 1000

with tf.Session() as sess:
    sess.run(init_op)
    writer = tf.summary.FileWriter(output_dir, sess.graph)
    for i in range(training_steps):
        batch_img_features, batch_sentence_ids, batch_weights, _ = caption_data.next_batch(hps.batch_size)
        input_vals = (batch_img_features, batch_sentence_ids, batch_weights, hps.keep_prob)
        feed_dict = dict(zip(placeholders, input_vals))
        fetches = [global_step, loss, accuracy, train_op]
        should_log = (i+1) % hps.log_frequent == 0
        should_save = (i+1) % hps.save_frequent == 0
        
        if should_log:
            fetches += [summary_op]
        
        outputs = sess.run(fetches, feed_dict = feed_dict)
        global_step_val, loss_val, accuracy_val = outputs[0:3]
        if should_log:
            summary_str = outputs[-1]
            writer.add_summary(summary_str, global_step_val)
            logging.info("Step: %5d, loss: %3.3f, accuracy: %3.3f" % (global_step_val, loss_val, accuracy_val))
        
        if should_save:
            model_save_file = os.path.join(output_dir, "image_caption")
            logging.info("Step: %5d, model saved" % global_step_val)
            saver.save(sess, model_save_file, global_step = global_step_val)

INFO:tensorflow:Step:    10, loss: 9.245, accuracy: 0.000
INFO:tensorflow:Step:    20, loss: 8.952, accuracy: 0.000
INFO:tensorflow:Step:    30, loss: 8.704, accuracy: 0.015
INFO:tensorflow:Step:    40, loss: 8.297, accuracy: 0.029
INFO:tensorflow:Step:    50, loss: 7.675, accuracy: 0.055
INFO:tensorflow:Step:    60, loss: 7.159, accuracy: 0.075
INFO:tensorflow:Step:    70, loss: 6.418, accuracy: 0.114
INFO:tensorflow:Step:    80, loss: 6.244, accuracy: 0.129
INFO:tensorflow:Step:    90, loss: 5.916, accuracy: 0.143
INFO:tensorflow:Step:   100, loss: 5.982, accuracy: 0.130
INFO:tensorflow:Step:   100, model saved
INFO:tensorflow:Step:   110, loss: 5.912, accuracy: 0.130
INFO:tensorflow:Step:   120, loss: 5.845, accuracy: 0.151
INFO:tensorflow:Step:   130, loss: 5.567, accuracy: 0.155
INFO:tensorflow:Step:   140, loss: 5.730, accuracy: 0.133
INFO:tensorflow:Step:   150, loss: 5.777, accuracy: 0.117
INFO:tensorflow:Step:   160, loss: 5.790, accuracy: 0.123
INFO:tensorflow:Step:   170, lo